In [79]:
import pandas as pd

In [80]:
path = "data/Crime_Incidents_in_2024.csv"

In [81]:
df = pd.read_csv(path)

In [82]:
df.head()

,CCN,REPORT_DATE,SHIFT,METHOD,OFFENSE,BLOCK,XBLOCK,YBLOCK,WARD,ANC,...,NEIGHBORHOOD_CLUSTER,BLOCK_GROUP,CENSUS_TRACT,VOTING_PRECINCT,LATITUDE,LONGITUDE,BID,START_DATE,END_DATE,OBJECTID
0,24423221,2024/09/24 18:40:56+00,DAY,OTHERS,THEFT/OTHER,300 - 399 BLOCK OF RHODE ISLAND AVENUE NE,399887.24,139069.89,5.0,5F,...,Cluster 21,009203 2,9203.0,Precinct 74,38.919499,-77.001300,NaN,2024/09/04 16:43:00+00,2024/09/04 16:45:00+00,737857565
1,24009631,2024/01/20 09:21:04+00,MIDNIGHT,OTHERS,THEFT/OTHER,800 - 899 BLOCK OF R STREET NW,397965.67,138304.93,2.0,2G,...,Cluster 7,004901 1,4901.0,Precinct 21,38.912606,-77.023456,NaN,2024/01/20 07:50:00+00,2024/01/20 09:40:00+00,737857566
2,24009706,2024/01/20 17:29:27+00,DAY,OTHERS,THEFT F/AUTO,1000 - 1199 BLOCK OF OTIS STREET NE,400695.73,140731.99,5.0,5B,...,Cluster 20,009504 1,9504.0,Precinct 68,38.934472,-76.991976,NaN,2024/01/18 03:00:00+00,2024/01/18 23:00:00+00,737857567
3,24421835,2024/05/21 12:01:09+00,DAY,OTHERS,THEFT/OTHER,600 - 623 BLOCK OF 6TH STREET NE,400134.57,136653.33,6.0,6C,...,Cluster 25,008301 1,8301.0,Precinct 84,38.897730,-76.998449,NaN,2024/05/14 04:20:00+00,2024/05/14 17:07:00+00,737857575
4,24422596,2024/08/02 05:42:27+00,MIDNIGHT,OTHERS,THEFT/OTHER,600 - 699 BLOCK OF ELLIOTT STREET NE,401170.24,136698.95,6.0,6A,...,Cluster 25,008001 1,8001.0,Precinct 81,38.898140,-76.986510,NaN,2024/07/16 16:14:00+00,2024/07/16 16:15:00+00,737857576


In [83]:
df.columns

Index(['CCN', 'REPORT_DATE', 'SHIFT', 'METHOD', 'OFFENSE', 'BLOCK', 'XBLOCK',
       'YBLOCK', 'WARD', 'ANC', 'DISTRICT', 'PSA', 'NEIGHBORHOOD_CLUSTER',
       'BLOCK_GROUP', 'CENSUS_TRACT', 'VOTING_PRECINCT', 'LATITUDE',
       'LONGITUDE', 'BID', 'START_DATE', 'END_DATE', 'OBJECTID'],
      dtype='object')

In [84]:
offense_df = df[['OFFENSE']].drop_duplicates().reset_index(drop=True)
offense_df.rename(columns={'OFFENSE': 'offense_type'}, inplace=True)
offense_df['offense_id'] = offense_df.index + 1
offense_df = offense_df[['offense_id', 'offense_type']]
offense_df.to_csv('offense.csv', index=False)


In [85]:
method_df = df[['METHOD']].drop_duplicates().reset_index(drop=True)
method_df['method_id'] = method_df.index + 1
method_df.rename(columns={'METHOD': 'method_type'}, inplace=True)
method_df = method_df[['method_id', 'method_type']]
method_df.to_csv('method.csv', index=False)


In [86]:
# Create location DataFrame
location_df = df[['WARD', 'DISTRICT', 'PSA', 'ANC', 'NEIGHBORHOOD_CLUSTER']].drop_duplicates().reset_index(drop=True)

# Assign and convert location_id to integer
location_df['location_id'] = location_df.index + 1
location_df['location_id'] = location_df['location_id'].astype(int)  # Force integer type

# Reorder columns if needed
location_df = location_df[['location_id', 'WARD', 'DISTRICT', 'PSA', 'ANC', 'NEIGHBORHOOD_CLUSTER']]

# Save to CSV
location_df.to_csv('location.csv', index=False)

In [93]:
location_df = df[['WARD', 'DISTRICT', 'PSA', 'ANC', 'NEIGHBORHOOD_CLUSTER']].drop_duplicates().reset_index(drop=True)
location_df['location_id'] = location_df.index + 1

# Merge back the location_id into the main df based on those 5 geographic fields
df = df.merge(location_df, on=['WARD', 'DISTRICT', 'PSA', 'ANC', 'NEIGHBORHOOD_CLUSTER'], how='left')

block_df = df[['XBLOCK', 'YBLOCK', 'location_id']].drop_duplicates()
block_df.rename(columns={'XBLOCK': 'x_block', 'YBLOCK': 'y_block'}, inplace=True)
block_df.to_csv('block_group.csv', index=False)


In [88]:
# Map method_id
df = df.merge(method_df, left_on='METHOD', right_on='method_type', how='left')

# Map offense_id
df = df.merge(offense_df, left_on='OFFENSE', right_on='offense_type', how='left')

# Assign location_id (merge from location table)
df = df.merge(location_df, on=['WARD', 'DISTRICT', 'PSA', 'ANC', 'NEIGHBORHOOD_CLUSTER'], how='left')

# Create the final crime_incident CSV
incident_df = df[['CCN', 'REPORT_DATE', 'START_DATE', 'END_DATE', 'SHIFT', 'offense_id', 'method_id']]
incident_df.rename(columns={
    'REPORT_DATE': 'report_date',
    'START_DATE': 'start_date',
    'END_DATE': 'end_date',
    'SHIFT': 'shift'
}, inplace=True)
incident_df = incident_df.drop_duplicates(subset=['CCN'])
incident_df.to_csv('crime_incident.csv', index=False)


/var/folders/p4/1lvg1px941165j7l1h_yj9j80000gn/T/ipykernel_24294/2094713905.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  incident_df.rename(columns={


In [91]:
crime_location_df = df[['CCN', 'location_id', 'LATITUDE', 'LONGITUDE']]
crime_location_df.rename(columns={
    'LATITUDE': 'latitude',
    'LONGITUDE': 'longitude'
}, inplace=True)
crime_location_df = crime_location_df.drop_duplicates(subset=['CCN'])
crime_location_df.to_csv('crime_location.csv', index=False)


/var/folders/p4/1lvg1px941165j7l1h_yj9j80000gn/T/ipykernel_24294/1058803027.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crime_location_df.rename(columns={
